In [3]:
import os
import json
import pandas as pd
import traceback

In [4]:
from langchain.chat_models import ChatOpenAI


In [5]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [6]:
KEY=os.getenv("OPENAI_API_KEY")

In [ ]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)